In [ ]:
appname = "dog food spoiling"

# Look into https://spark.apache.org/downloads.html for the latest version
spark_mirror = "https://mirrors.sonic.net/apache/spark"
spark_version = "3.3.1"
hadoop_version = "3"

# Install Java 8 (Spark does not work with newer Java versions)
! apt-get update
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download and extract Spark binary distribution
! rm -rf spark-{spark_version}-bin-hadoop{hadoop_version}.tgz spark-{spark_version}-bin-hadoop{hadoop_version}
! wget -q {spark_mirror}/spark-{spark_version}/spark-{spark_version}-bin-hadoop{hadoop_version}.tgz
! tar xzf spark-{spark_version}-bin-hadoop{hadoop_version}.tgz

# The only 2 environment variables needed to set up Java and Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{spark_version}-bin-hadoop{hadoop_version}"

# Set up the Spark environment based on the environment variable SPARK_HOME 
! pip install -q findspark
import findspark
findspark.init()

# Get the Spark session object (basic entry point for every operation)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(appname).master("local[*]").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,259 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:13 http://security.u

In [ ]:
#DOWNLOAD AND READ THE FILES
from google.colab import files 
  
dog_food = files.upload()

Saving dog_food.csv to dog_food.csv


In [ ]:
df = spark.read.options(inferSchema=True, header=True).csv('dog_food.csv')
df.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [ ]:
#VECTORASSEMBLER ASSEMBLES ALL THE FEATURES INTO ONE VECTOR FROM MULTIPLE COLUMNS THAT CONTAIN TYPE DOUBLE. 
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol = 'features')
output = assembler.transform(df)
compared_data = output.select("features","Spoiled")
df2 = compared_data.withColumnRenamed("Spoiled","label")
df2.printSchema()


root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)



In [ ]:
df2.show()

+-------------------+-----+
|           features|label|
+-------------------+-----+
| [4.0,2.0,12.0,3.0]|  1.0|
| [5.0,6.0,12.0,7.0]|  1.0|
| [6.0,2.0,13.0,6.0]|  1.0|
| [4.0,2.0,12.0,1.0]|  1.0|
| [4.0,2.0,12.0,3.0]|  1.0|
|[10.0,3.0,13.0,9.0]|  1.0|
| [8.0,5.0,14.0,5.0]|  1.0|
| [5.0,8.0,12.0,8.0]|  1.0|
| [6.0,5.0,12.0,9.0]|  1.0|
| [3.0,3.0,12.0,1.0]|  1.0|
| [9.0,8.0,11.0,3.0]|  1.0|
|[1.0,10.0,12.0,3.0]|  1.0|
|[1.0,5.0,13.0,10.0]|  1.0|
|[2.0,10.0,12.0,6.0]|  1.0|
|[1.0,10.0,11.0,4.0]|  1.0|
| [5.0,3.0,12.0,2.0]|  1.0|
| [4.0,9.0,11.0,8.0]|  1.0|
| [5.0,1.0,11.0,1.0]|  1.0|
|[4.0,9.0,12.0,10.0]|  1.0|
| [5.0,8.0,10.0,9.0]|  1.0|
+-------------------+-----+
only showing top 20 rows



In [ ]:
#IMPORT THE TREE METHODS
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier,
                                       DecisionTreeClassifier)

In [ ]:
#TRAIN THE DATA AND CREATE THE MODEL
train, test = df2.randomSplit([0.7, 0.3])

In [ ]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees = 100)
gbt = GBTClassifier()

In [ ]:
test.show()

+-------------------+-----+
|           features|label|
+-------------------+-----+
| [1.0,1.0,12.0,2.0]|  1.0|
|  [1.0,2.0,9.0,1.0]|  0.0|
|  [1.0,3.0,8.0,3.0]|  0.0|
|  [1.0,3.0,9.0,8.0]|  0.0|
|[1.0,5.0,12.0,10.0]|  1.0|
|  [1.0,6.0,7.0,8.0]|  0.0|
|  [1.0,6.0,8.0,1.0]|  0.0|
|  [1.0,6.0,8.0,3.0]|  0.0|
| [1.0,8.0,7.0,10.0]|  0.0|
|  [1.0,8.0,8.0,7.0]|  0.0|
| [1.0,8.0,12.0,1.0]|  1.0|
|  [1.0,9.0,7.0,4.0]|  0.0|
|[1.0,10.0,11.0,4.0]|  1.0|
|  [2.0,2.0,9.0,8.0]|  0.0|
|  [2.0,3.0,9.0,3.0]|  0.0|
| [2.0,4.0,13.0,4.0]|  1.0|
|  [2.0,5.0,8.0,3.0]|  0.0|
|  [2.0,5.0,8.0,3.0]|  0.0|
|  [2.0,6.0,6.0,9.0]|  0.0|
| [2.0,6.0,8.0,10.0]|  0.0|
+-------------------+-----+
only showing top 20 rows



In [ ]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

In [ ]:
dtc_preds = dtc_model.transform(test)
rfc_preds = rfc_model.transform(test)
gbt_preds = gbt_model.transform(test)

In [ ]:
dtc_preds.show()

+-------------------+-----+-------------+-----------+----------+
|           features|label|rawPrediction|probability|prediction|
+-------------------+-----+-------------+-----------+----------+
| [1.0,1.0,12.0,2.0]|  1.0|   [0.0,62.0]|  [0.0,1.0]|       1.0|
|  [1.0,2.0,9.0,1.0]|  0.0|    [6.0,0.0]|  [1.0,0.0]|       0.0|
|  [1.0,3.0,8.0,3.0]|  0.0|  [183.0,0.0]|  [1.0,0.0]|       0.0|
|  [1.0,3.0,9.0,8.0]|  0.0|  [183.0,0.0]|  [1.0,0.0]|       0.0|
|[1.0,5.0,12.0,10.0]|  1.0|   [0.0,62.0]|  [0.0,1.0]|       1.0|
|  [1.0,6.0,7.0,8.0]|  0.0|  [183.0,0.0]|  [1.0,0.0]|       0.0|
|  [1.0,6.0,8.0,1.0]|  0.0|   [26.0,0.0]|  [1.0,0.0]|       0.0|
|  [1.0,6.0,8.0,3.0]|  0.0|  [183.0,0.0]|  [1.0,0.0]|       0.0|
| [1.0,8.0,7.0,10.0]|  0.0|  [183.0,0.0]|  [1.0,0.0]|       0.0|
|  [1.0,8.0,8.0,7.0]|  0.0|  [183.0,0.0]|  [1.0,0.0]|       0.0|
| [1.0,8.0,12.0,1.0]|  1.0|   [0.0,62.0]|  [0.0,1.0]|       1.0|
|  [1.0,9.0,7.0,4.0]|  0.0|  [183.0,0.0]|  [1.0,0.0]|       0.0|
|[1.0,10.0,11.0,4.0]|  1.

In [ ]:
#IMPORT THE EVALUATORS
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [ ]:
print(evaluator.explainParam('metricName'))

metricName: metric name in evaluation (f1|accuracy|weightedPrecision|weightedRecall|weightedTruePositiveRate| weightedFalsePositiveRate|weightedFMeasure|truePositiveRateByLabel| falsePositiveRateByLabel|precisionByLabel|recallByLabel|fMeasureByLabel| logLoss|hammingLoss) (default: f1, current: accuracy)


In [ ]:
print(f'DTC: {evaluator.evaluate(dtc_preds)}\t Features Importance: {dtc_model.featureImportances}') 
print(f'RFC: {evaluator.evaluate(rfc_preds)} Features Importance: {rfc_model.featureImportances}')
print(f'GBT: {evaluator.evaluate(gbt_preds)}\t Features Importance: {gbt_model.featureImportances}')

DTC: 0.9757575757575757	 Features Importance: (4,[0,1,2,3],[0.005203898066578417,0.018633043075897186,0.9448457659422271,0.03131729291529735])
RFC: 0.9818181818181818 Features Importance: (4,[0,1,2,3],[0.02809578879732174,0.02447173774778857,0.9152241173745608,0.032208356080328945])
GBT: 0.9757575757575757	 Features Importance: (4,[0,1,2,3],[0.03496010791356495,0.024365435757374924,0.8990271852620919,0.0416472710669682])


In [ ]:
evaluator.evaluate(gbt_preds, { evaluator.metricName: 'f1'})

0.9755959300716333

Our aim was to determine what element in the dog food was causing it to spoil. With the .featureimportance command, we can find out what weight each column was having on the Spoiled column. As can be seen, in all cases column three always has the most weight, indicating that element C in the dog food is causing it to spoil.